In [2]:
DATA_PATH = "WikiSQL/data/"

In [1]:
import json
import numpy as np
import pandas as pd

In [7]:
def read_jsonl(jsonl):
    for line in open(jsonl, encoding="utf8"):
        sample = json.loads(line.rstrip())
        yield sample

In [4]:
with open(DATA_PATH + "dev.jsonl", 'r') as f_dev:
   dev = list(f_dev)

with open(DATA_PATH + "dev.tables.jsonl", 'r') as f_devtable:
   devtable = list(f_devtable)

json.loads(dev[0]).keys(), json.loads(devtable[0]).keys()

(dict_keys(['phase', 'table_id', 'question', 'sql']),
 dict_keys(['header', 'page_title', 'types', 'id', 'section_title', 'caption', 'rows', 'name']))

In [5]:
json.loads(devtable[0])['header']

['Player',
 'No.',
 'Nationality',
 'Position',
 'Years in Toronto',
 'School/Club Team']

In [17]:
for i in range(5):  
   print(json.loads(dev[i]))

{'phase': 1, 'table_id': '1-10015132-11', 'question': 'What position does the player who played for butler cc (ks) play?', 'sql': {'sel': 3, 'conds': [[5, 0, 'Butler CC (KS)']], 'agg': 0}}
{'phase': 1, 'table_id': '1-10015132-11', 'question': 'How many schools did player number 3 play at?', 'sql': {'sel': 5, 'conds': [[1, 0, '3']], 'agg': 3}}
{'phase': 1, 'table_id': '1-10015132-11', 'question': 'What school did player number 21 play for?', 'sql': {'sel': 5, 'conds': [[1, 0, '21']], 'agg': 0}}
{'phase': 1, 'table_id': '1-10015132-11', 'question': 'Who is the player that wears number 42?', 'sql': {'sel': 0, 'conds': [[1, 0, '42']], 'agg': 0}}
{'phase': 1, 'table_id': '1-10015132-11', 'question': 'What player played guard for toronto in 1996-97?', 'sql': {'sel': 0, 'conds': [[3, 0, 'Guard'], [4, 0, '1996-97']], 'agg': 0}}


In [27]:
for i in range(5):  
   print(json.loads(devtable[i]))

{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-11', 'section_title': 'L', 'caption': 'L', 'rows': [['Antonio Lang', '21', 'United States', 'Guard-Forward', '1999-2000', 'Duke'], ['Voshon Lenard', '2', 'United States', 'Guard', '2002-03', 'Minnesota'], ['Martin Lewis', '32, 44', 'United States', 'Guard-Forward', '1996-97', 'Butler CC (KS)'], ['Brad Lohaus', '33', 'United States', 'Forward-Center', '1996', 'Iowa'], ['Art Long', '42', 'United States', 'Forward-Center', '2002-03', 'Cincinnati'], ['John Long', '25', 'United States', 'Guard', '1996-97', 'Detroit'], ['Kyle Lowry', '3', 'United States', 'Guard', '2012-Present', 'Villanova']], 'name': 'table_10015132_11'}
{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'types

In [11]:
# table schema
def get_schema(tables):
    schema, headers, colTypes, naturalMap = {}, {}, {}, {}
    for table in tables:
        values = [set() for _ in range(len(table["header"]))]
        for row in table["rows"]:
            for i, value in enumerate(row):
                values[i].add(str(value).lower())
        columns = {column: values[i] for i, column in enumerate(table["header"])}

        trans = {"text": "string", "real": "real"}
        colTypes[table["id"]] = {col:trans[ty] for ty, col in zip(table["types"], table["header"])}
        schema[table["id"]] = columns
        naturalMap[table["id"]] = {col: col for col in columns}
        headers[table["id"]] = table["header"]
        
        break

    return schema, headers, colTypes, naturalMap

In [12]:
get_schema(read_jsonl(DATA_PATH + "dev.tables.jsonl"))

({'1-10015132-11': {'Player': {'antonio lang',
    'art long',
    'brad lohaus',
    'john long',
    'kyle lowry',
    'martin lewis',
    'voshon lenard'},
   'No.': {'2', '21', '25', '3', '32, 44', '33', '42'},
   'Nationality': {'united states'},
   'Position': {'forward-center', 'guard', 'guard-forward'},
   'Years in Toronto': {'1996',
    '1996-97',
    '1999-2000',
    '2002-03',
    '2012-present'},
   'School/Club Team': {'butler cc (ks)',
    'cincinnati',
    'detroit',
    'duke',
    'iowa',
    'minnesota',
    'villanova'}}},
 {'1-10015132-11': ['Player',
   'No.',
   'Nationality',
   'Position',
   'Years in Toronto',
   'School/Club Team']},
 {'1-10015132-11': {'Player': 'string',
   'No.': 'string',
   'Nationality': 'string',
   'Position': 'string',
   'Years in Toronto': 'string',
   'School/Club Team': 'string'}},
 {'1-10015132-11': {'Player': 'Player',
   'No.': 'No.',
   'Nationality': 'Nationality',
   'Position': 'Position',
   'Years in Toronto': 'Years in